In [1]:
import time
import calendar
import codecs
import datetime
import json
import sys
import gzip
import string
import glob
import re
import os
import numpy as np
from os import listdir
from os.path import isfile, join
import csv
import networkx as nx
from demjson import decode
import pandas as pd
import random

from nltk.corpus import stopwords

In [2]:
globalTweetCounter = 0

timeFormat = "%Y-%m-%dT%H:%M:%S.%fZ"

In [55]:
countries=['SAfrica-community-relevant-restricted.json']

for m in range(len(countries)):
    file_string=countries[m]
    mypath='/Volumes/pond/Temp/twitter/'+file_string
    G=nx.Graph()
    tweetList = []
    timeList = []
    userList = []
    
    with open(mypath) as f1:
        for line in f1:
           if random.random() < 1.1: # < 1 if you want less data
                tweetObj = json.loads(line)
                currentTime = datetime.datetime.strptime(tweetObj['postedTime'], timeFormat)
                id2=int(re.findall('^.*:([0-9]+)$',str(tweetObj['actor']['id']))[0])
                G.add_node(id2, parent=True, child=False)
                for ui in tweetObj['twitter_entities']['user_mentions']:
                    id1=ui['id']
                    G.add_node(id1, child=True)
                    G.add_edge(id1,id2, posted=currentTime, message=tweetObj['body'])
                    #print id1, id2
                try:
                    if (not tweetObj['body'].lower().startswith("rt")):
                        # Increment tweet count
                        globalTweetCounter += 1
                        tweetList.append(tweetObj['body'].lower())
                        timeList.append(currentTime)
                        userList.append(tweetObj['actor']['id'])
                        #print globalTweetCounter, tweetObj['body'].lower()
                except:
                    pass


In [56]:
print np.min(timeList)
print np.max(timeList)
split = datetime.datetime(2014, 5, 25) #train/test split

2014-03-15 04:26:31
2014-06-13 22:05:22


In [57]:
print G.number_of_edges(), G.number_of_nodes(), len(timeList)

173629 26669 90502


In [58]:
for node in G.nodes():
    nTrain = 0
    nTest = 0
    for nbr in G.neighbors(node):
        if G.edge[node][nbr]['posted'] < split:
            nTrain += 1
        else:
            nTest += 1
    if nTrain < 3 or nTest < 3:
        G.remove_node(node)

In [59]:
print G.number_of_edges(), G.number_of_nodes()

21498 1267


In [48]:
#for u, v in G.edges(20521828):
#    print G.edge[u][v]['message'], v

On @SABCNewsOnline - @helenzille speaks on the #LwandleEvictions http://t.co/erVER2EYwo 34635844
RT @deondelange: @helenzille If someone defaced a DA poster - an illegal act - you would be hopping mad. Don't encourage criminality on a p… 128843902
RT @helenzille: Re the ballots that were found:  auditing has established that they WERE counted before they were "dumped".  No fraud suspe… 151027855
@sandafresha @helenzille @aperiozar Eskom announces load shedding http://t.co/WSZaohxJJ3 350257318
@ALETTAHA @aperiozar @prettylilnaz @helenzille yep, agree - but still should have pelted him with rotten eggs before leaving! #VoteDA 721092824
Some1 need to repossessed @helenzille &amp; all her #DA supporters their fones or suspend their twitter account until  #Inauguration2014 348356829
@da_news @helenzille Got an uneasy feeling ANC have rigged the win in West Cape. Open ballot boxes, pre checked Regional ballots etc. 22249769
@Eusebius check @helenzille TL, she makes it clear @MaimaneAM was be

In [51]:
for nbr in nx.common_neighbors(G, 20521828, 311002833):
    print nbr
    
j = nx.jaccard_coefficient(G, [(20521828, 311002833)])
for u,v,p in j:
    print u,v,p

#print G.edges(20521828)
#print G.edge[81068057][463026107]

23594033
45189202
117149296
77338811
188031826
2447647992
1026960121
337554845
155568211
467318339
20433284
18759465
2235646464
2301806896
2265533133
1534577533
2218609787
25645943
266228663
249846653
539943816
597387389
452137334
2210884433
887160097
21098456
191795989
256177204
173242630
449149707
20521828 311002833 0.032154340836


In [52]:
counter=0
for n, nbrs in G.adjacency_iter():
    #print G.node[n]
    counter += 1
    #for nbr in nbrs:
        #print("\t%d" % nbr)
print counter

1057


In [53]:
print G.node[20521828]

{'parent': True, 'child': True}
